<a href="https://colab.research.google.com/github/sakethrammamidi/Stock-Market-Analysis/blob/main/Stock_Market_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance

Below is how we can connect real-time stock market data using the yfinance API:

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# Calculate start and end dates for the last 3 months
end_date = datetime.now()
start_date = end_date - timedelta(days=90)

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2024-02-01  183.990005  186.949997  183.820007  186.860001  186.621872   
       2024-02-02  179.860001  187.330002  179.250000  185.850006  185.613159   
       2024-02-05  188.149994  189.250000  185.839996  187.679993  187.440811   
       2024-02-06  186.860001  189.309998  186.770004  189.300003  189.058762   
       2024-02-07  190.639999  191.050003  188.610001  189.410004  189.168625   

                      Volume  
Ticker Date                   
AAPL   2024-02-01   64885400  
       2024-02-02  102518000  
       2024-02-05   69668800  
       2024-02-06   43490800  
       2024-02-07   53439000  


In the above code, we first imported the necessary Python libraries and downloaded the historical stock price data for four companies: Apple, Microsoft, Netflix, and Google, for the last three months.

In this dataset, the Date column is the index column in the DataFrame. We need to reset the index before moving forward:

In [ ]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2024-02-01  183.990005  186.949997  183.820007  186.860001   
1   AAPL 2024-02-02  179.860001  187.330002  179.250000  185.850006   
2   AAPL 2024-02-05  188.149994  189.250000  185.839996  187.679993   
3   AAPL 2024-02-06  186.860001  189.309998  186.770004  189.300003   
4   AAPL 2024-02-07  190.639999  191.050003  188.610001  189.410004   

    Adj Close     Volume  
0  186.621872   64885400  
1  185.613159  102518000  
2  187.440811   69668800  
3  189.058762   43490800  
4  189.168625   53439000  


Now let's have a look at the performance in the stock market of all the companies:


In [ ]:
import plotly.express as px
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

Now let's look at the faceted area chart, which makes it easy to compare the performance of different companies and identify similarities or differences in their stock price movements:

In [ ]:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

Now let's analyze moving averages, which provide a useful way to identify trends and patterns in each company's stock price movements over a period of time:



In [ ]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
57  169.545999  169.653499
58  169.031000  169.482499
59  168.306000  169.373499
60  168.387000  169.546999
61  168.481999  169.621500

[62 rows x 2 columns]
Moving Averages for GOOG
           MA10      MA20
186         NaN       NaN
187         NaN       NaN
188         NaN       NaN
189         NaN       NaN
190         NaN       NaN
..          ...       ...
243  158.134001  156.6050
244  157.850002  156.9055
245  159.300002  157.9770
246  160.457001  158.5470
247  161.321001  158.9855

[62 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
62          NaN         NaN
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
..          ...         ...
119  411.086996  416.850499
120  408.197998  41

Now here's how to visualize the moving averages of all companies:

In [ ]:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

Let us now analyze the volatility of all companies.

In [ ]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

Now let's analyze the correlation between the stock prices of Apple and Microsoft:


In [ ]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',
                 title='Correlation between Apple and Microsoft')
fig.show()